In [1]:
import numpy as np

In [2]:
test_matrix = np.random.random((31_650, 768)).astype(np.float32)
test_matrix.shape

(31650, 768)

In [3]:
print(f"{round(test_matrix.nbytes / 1024 / 1024, 2)} MB")

92.72 MB


In [4]:
query_embed = np.random.random(768).astype(np.float32)
query_embed[0:10]

array([0.9145591 , 0.9032284 , 0.44441798, 0.5654556 , 0.87278974,
       0.01689521, 0.30128086, 0.26281437, 0.189137  , 0.27175257],
      dtype=float32)

## Test Algo (numpy)

Algorithm must return both `idx` of highest `top_k=3` dot products and their `scores`.


In [5]:
result = query_embed @ test_matrix.T
result

array([184.45253, 185.08842, 191.39977, ..., 193.30887, 188.85185,
       178.51215], dtype=float32)

In [6]:
k = 3

idx = np.argpartition(result, -k)[-k:]
idx = idx[np.argsort(result[idx])[::-1]]
idx

array([12209,  6855, 16983])

In [7]:
score = result[idx]
score

array([204.00552, 203.6571 , 203.14044], dtype=float32)

## Benchmark


In [8]:
def fast_dot_product(query, matrix):
    dot_products = query @ matrix.T

    idx = np.argpartition(dot_products, -k)[-k:]
    idx = idx[np.argsort(dot_products[idx])[::-1]]

    score = dot_products[idx]

    return idx, score

In [9]:
fast_dot_product(query_embed, test_matrix)

(array([12209,  6855, 16983]),
 array([204.00552, 203.6571 , 203.14044], dtype=float32))

In [10]:
%%timeit

_ = fast_dot_product(query_embed, test_matrix)

1.07 ms ± 50.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Hamming Distance


In [17]:
test_matrix_hamming = np.where(test_matrix > 0.5, 0, 1).astype(np.uint8)
query_hamming = np.where(query_embed > 0.5, 0, 1).astype(np.uint8)

In [18]:
print(f"{round(test_matrix_hamming.nbytes / 1024 / 1024, 2)} MB")

23.18 MB


https://stackoverflow.com/a/32731247


In [29]:
def fast_hamming(query, matrix):
    hamming_distances = np.count_nonzero(matrix != query, axis=1)

    idx = np.argpartition(hamming_distances, -k)[-k:]
    idx = idx[np.argsort(hamming_distances[idx])[::-1]]

    score = hamming_distances[idx]

    return idx, score

In [30]:
fast_hamming(query_hamming, test_matrix_hamming)

(array([ 1654,  2336, 26247]), array([434, 434, 434]))

In [31]:
%%timeit

_ = fast_hamming(query_embed, test_matrix)

11.6 ms ± 231 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
